<a href="https://colab.research.google.com/github/FullPowerMore02/data-mining/blob/main/project06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install newspaper3k
!pip install pandas
!pip install openpyxl
!pip install tabulate
!pip install bs4
!pip install urllib3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=6e4fd07f52ecd4f1b3d46e7343abc0d90e6502dcfabd6121f7e0d67b7227871a
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58df48c00b9a31d638005143b5b3ac62c3d25fb1447f23
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=b216a47b37810888ab1bd457d68a07abd9787e25a4b4c08b2e25fb945ff31e28
  Stored in directory: /root/.cache/pip/wheels/97/02/e7/a1ff1760e12bdbaab0ac824fae5c1bc9

In [2]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from datetime import datetime, timedelta
from openpyxl import Workbook
import pandas as pd
import newspaper

In [11]:
def get_article_content(link):
    article_client = urlopen(link)
    article_page = soup(article_client, "html.parser")
    article_content = article_page.find("div", {"class": "article-body"})

    if article_content:
        return article_content.text.strip()
    else:
        return "No article content found."

def filter_news(news_list, search_keywords):
    filtered_news = []
    for news in news_list:
        title = news.title.text
        if any(keyword in title.lower() for keyword in search_keywords):
            filtered_news.append(news)
    return filtered_news

def news():
    search_keywords = ["cryptocurrency", "bitcoin", "blockchain", "ethereum", "binance", "xrp", "litecoin"]
    url_keywords = "+OR+".join(search_keywords)

    start_date = datetime(2023, 7, 13)
    end_date = datetime(2023, 7, 14)

    current_date = start_date

    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["Title", "Link", "Publication Date", "Article Content"])

    while current_date <= end_date:
        search_date = current_date.strftime("%Y-%m-%d")

        url = f"https://news.google.com/rss/search?q={search_date}%20{url_keywords}&hl=en-US&gl=US&ceid=US%3Aen"

        client = urlopen(url)
        xml_page = client.read()
        client.close()

        page = soup(xml_page, "xml")
        news_list = page.findAll("item")

        news_list = filter_news(news_list, search_keywords)

        for news in news_list:
            title = news.title.text
            link = news.link.text

            article_content = get_article_content(link)

            sheet.append([title, link, current_date.strftime("%Y-%m-%d"), article_content])

        current_date += timedelta(days=1)

    workbook.save("news_data02.xlsx")

if __name__ == "__main__":
    news()



In [12]:
df = pd.read_excel('/content/news_data02.xlsx')
df


,Title,Link,Publication Date,Article Content
0,Ripple Labs notches landmark win in SEC case o...,https://news.google.com/rss/articles/CBMicGh0d...,2023-07-13,No article content found.
1,XRP coin surges after judge delivers a huge wi...,https://news.google.com/rss/articles/CBMidWh0d...,2023-07-13,No article content found.
2,Ripple’s XRP Token Surges 96% After Partial Vi...,https://news.google.com/rss/articles/CBMif2h0d...,2023-07-13,No article content found.
3,Analysis: Partnering with Coinbase could hinde...,https://news.google.com/rss/articles/CBMibWh0d...,2023-07-13,No article content found.
4,Argentina's First Bitcoin Futures Contract Off...,https://news.google.com/rss/articles/CBMia2h0d...,2023-07-13,No article content found.
...,...,...,...,...
82,Gold back to $2100 in a hurry as Bitcoin threa...,https://news.google.com/rss/articles/CBMia2h0d...,2023-07-14,No article content found.
83,Whales & sharks increasingly accumulate stable...,https://news.google.com/rss/articles/CBMiP2h0d...,2023-07-14,No article content found.
84,"Next Cryptocurrency To Explode 2023: Dogecoin,...",https://news.google.com/rss/articles/CBMikgFod...,2023-07-14,No article content found.
85,Bitcoin (BTC/USD) Price Analysis Report – 4 Ju...,https://news.google.com/rss/articles/CBMiXWh0d...,2023-07-14,No article content found.


In [ ]:
def get_destination_link(link):
    try:
        page_client = urlopen(link)
        page_content = page_client.read().decode('utf-8')
        page_soup = soup(page_content, 'html.parser')
        destination_link = page_soup.find('a')['href']
        return destination_link
    except:
        return np.nan

# อ่านไฟล์ Excel
df_links = pd.read_excel("/content/news_data02.xlsx")

# สร้างรายการเพื่อเก็บผลลัพธ์ของบทความ
results = []

# ใช้คอลัมน์ "Link" เพื่อดึง Destination Link และเนื้อหาของบทความ
for index, row in df_links.iterrows():
    link = row["Link"]
    destination_link = get_destination_link(link)

    print("Destination Link:", destination_link)

    try:
        article = newspaper.Article(link)
        article.download()
        article.parse()

        title = article.title
        news_content = article.text
        authors = article.authors
        publish_date = article.publish_date
        top_image = article.top_image

        print("Title:", title)
        print("News Content:", news_content)
        print("Authors:", authors)
        print("Publish Date:", publish_date)
        print("Top Image URL:", top_image)

        # แปลงวันที่และเวลาให้เป็นข้อมูลปกติ (timezone unaware)
        if publish_date is not None:
            publish_date = publish_date.replace(tzinfo=None)

        # เพิ่มผลลัพธ์ของบทความลงในรายการ
        results.append({
            "Destination Link": destination_link,
            "Title": title,
            "News Content": news_content,
            "Authors": authors,
            "Publish Date": publish_date,
            "Top Image URL": top_image
        })

    except Exception as e:
        print("Error occurred:", str(e))
        print("Skipping to next link...")
        # เพิ่มแถวผลลัพธ์ที่มีค่า NaN ในกรณีที่การดึงข้อมูลไม่สำเร็จ
        results.append({
            "Destination Link": np.nan,
            "Title": np.nan,
            "News Content": np.nan,
            "Authors": np.nan,
            "Publish Date": np.nan,
            "Top Image URL": np.nan
        })

    print("-----------------------")

# สร้าง DataFrame จากผลลัพธ์ของบทความ
df_results = pd.DataFrame(results)

# บันทึกลงในไฟล์ Excel
df_results.to_excel("article_results.xlsx", index=False)


In [16]:
df = pd.read_excel('/content/article_results.xlsx')
df


,Destination Link,Title,News Content,Authors,Publish Date,Top Image URL
0,https://www.reuters.com/legal/us-judge-says-se...,Ripple Labs notches landmark win in SEC case o...,Companies\n\nLaw Firms Ripple Labs Inc Follow\...,"['Jody Godoy', 'Thomson Reuters', 'Jody Godoy ...",2023-07-13 21:26:22.000,https://www.reuters.com/resizer/omhwmjXHMFRPru...
1,https://www.cnbc.com/2023/07/13/xrp-surges-aft...,XRP coin surges after judge delivers a huge wi...,In this article COIN\n\nBTC.CM=\n\nXRP.CM= Fol...,['Tanaya Macheel'],2023-07-13 16:11:28.000,https://image.cnbcfm.com/api/v1/image/10546926...
2,https://www.coindesk.com/markets/2023/07/13/ri...,Ripple’s XRP Token Surges 96% After Partial Vi...,The price action comes immediately after the D...,[],2023-07-13 16:44:39.948,https://www.coindesk.com/resizer/bK731G6FGsHAT...
3,https://www.reuters.com/technology/partnering-...,Analysis: Partnering with Coinbase could hinde...,"WASHINGTON, July 13 (Reuters) - Partnering wit...","['John Mccrank Michelle Price', 'John Mccrank'...",2023-07-13 20:05:32.000,https://www.reuters.com/resizer/E9nbnuYR2z5DpL...
4,https://www.coindesk.com/markets/2023/07/13/ar...,Argentina's First Bitcoin Futures Contract Off...,"In April, the country’s National Securities Co...",[],2023-07-13 15:44:06.425,https://www.coindesk.com/resizer/37y2gE2d-dBUY...
...,...,...,...,...,...,...
82,https://www.kitco.com/commentaries/2023-07-14/...,Gold back to $2100 in a hurry as Bitcoin threa...,I expected resistance at the 50-day MA in gold...,"['Http', 'Www.Facebook.Com Kitconews']",2023-07-14 14:06:00.000,https://www.kitco.com/commentaries/2023-07-14/...
83,https://www.investorsobserver.com/news/qm-news...,Whales & sharks increasingly accumulate stable...,Whales & sharks increasingly accumulate stable...,"['Thursday', 'July', 'Am', 'Invezz Via Quoteme...",NaT,https://s3.amazonaws.com/images.investorsobser...
84,https://newswatchtv.com/2023/07/14/next-crypto...,"Next Cryptocurrency To Explode 2023: Dogecoin,...",Meme coins have taken the crypto market by sto...,[],2023-07-14 06:23:14.000,https://newswatchtv.com/wp-content/uploads/202...
85,https://www.cryptoglobe.com/latest/2023/07/bit...,Bitcoin (BTC/USD) Price Analysis Report – 4 Ju...,Introduction\n\nThis report provides a compreh...,[],NaT,https://media.cryptoglobe.com/2023/06/price-an...


In [17]:
# อ่านไฟล์ Excel news_data02.xlsx
df_news = pd.read_excel("news_data02.xlsx")

# อ่านไฟล์ Excel article_results.xlsx
df_results = pd.read_excel("article_results.xlsx")

# สร้าง DataFrame ใหม่ที่รวมคอลัมน์ที่ต้องการจากทั้งสอง DataFrame
df_combined = pd.concat([df_news["Title"], df_news["Publication Date"], df_results["Destination Link"], df_results["News Content"]], axis=1)

# บันทึกลงในไฟล์ Excel
df_combined.to_excel("combined_results.xlsx", index=False)


In [18]:
df = pd.read_excel('/content/combined_results.xlsx')
df


,Title,Publication Date,Destination Link,News Content
0,Ripple Labs notches landmark win in SEC case o...,2023-07-13,https://www.reuters.com/legal/us-judge-says-se...,Companies\n\nLaw Firms Ripple Labs Inc Follow\...
1,XRP coin surges after judge delivers a huge wi...,2023-07-13,https://www.cnbc.com/2023/07/13/xrp-surges-aft...,In this article COIN\n\nBTC.CM=\n\nXRP.CM= Fol...
2,Ripple’s XRP Token Surges 96% After Partial Vi...,2023-07-13,https://www.coindesk.com/markets/2023/07/13/ri...,The price action comes immediately after the D...
3,Analysis: Partnering with Coinbase could hinde...,2023-07-13,https://www.reuters.com/technology/partnering-...,"WASHINGTON, July 13 (Reuters) - Partnering wit..."
4,Argentina's First Bitcoin Futures Contract Off...,2023-07-13,https://www.coindesk.com/markets/2023/07/13/ar...,"In April, the country’s National Securities Co..."
...,...,...,...,...
82,Gold back to $2100 in a hurry as Bitcoin threa...,2023-07-14,https://www.kitco.com/commentaries/2023-07-14/...,I expected resistance at the 50-day MA in gold...
83,Whales & sharks increasingly accumulate stable...,2023-07-14,https://www.investorsobserver.com/news/qm-news...,Whales & sharks increasingly accumulate stable...
84,"Next Cryptocurrency To Explode 2023: Dogecoin,...",2023-07-14,https://newswatchtv.com/2023/07/14/next-crypto...,Meme coins have taken the crypto market by sto...
85,Bitcoin (BTC/USD) Price Analysis Report – 4 Ju...,2023-07-14,https://www.cryptoglobe.com/latest/2023/07/bit...,Introduction\n\nThis report provides a compreh...
